In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas
from tqdm.notebook import tqdm

from data_loader import data_label_split
from data_loader import generate_data_set
from data_loader import dmso_taxol_ProfileBag, data_standardization

from set_experiment import mini_noise_signal_cv
from set_experiment import train
from set_experiment import test

from model import SmallDeepSet
from model import FullDeepSet, profile_AttSet

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize

import torch
import torch.utils.data as D 
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

In [2]:
drop_NA_data = pd.read_csv("moa_data_drop_NA.csv", index_col=0)
sf_drop_NA_data = drop_NA_data[["compound", "concentration",
                                "moa", "row ID", "Iteration (#2)", "COND",
                               "AreaShape_Area_Nuclei", "AreaShape_Compactness_Nuclei"]]

In [3]:
mini_data = pd.read_csv("mini_moa_data_drop_NA.csv",index_col=0)
# sf_drop_NA_data = mini_data[["compound", "concentration",
#                                 "moa", "row ID", "Iteration (#2)", "COND",
#                                "AreaShape_Area_Nuclei", "AreaShape_Compactness_Nuclei"]]

In [3]:

# bagData = dmso_taxol_ProfileBag(sf_drop_NA_data, 200, 50, 5, 0.5, "taxol", "DMSO", 0.5)
# dataloader = D.DataLoader(bagData, batch_size=1, shuffle=True)

In [5]:
# data = sf_drop_NA_data
# num_bag = 10
# bag_size_mean = 10
# bag_size_std = 0
# treatment = "taxol" 
# control = "DMSO"
# model = sf_deepset
# optimizer = optimizer
# splits = 4
# epochs = 1

# Deep Set Model

In [5]:
full_deepset = FullDeepSet(481)#.cuda()

In [6]:
sf_deepset = SmallDeepSet(481)#.cuda()

In [8]:
start = 90
end = 91
data = mini_data
num_bag = 60
bag_size_mean = 200
bag_size_std = 10
treatment = "taxol"
control = "DMSO"
batch_size = 8
model = profile_AttSet(481).cuda()#FullDeepSet("mean").cuda()
splits = 3
epochs = 10

In [5]:
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')



41,594 total parameters.
41,594 training parameters.


In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [10]:
results = mini_noise_signal_cv(start, end, data, num_bag,
                               bag_size_mean, bag_size_std, treatment, control, 
                               batch_size, model, optimizer, splits, epochs)

training at different percent:   0%|          | 0/1 [00:00<?, ?it/s]
3 fold cross validation: 0it [00:00, ?it/s]

Epoch: 0, Loss: 0.3519, Train error: 0.0125
At 0th fold testing: 


3 fold cross validation: 1it [00:18, 18.19s/it]

Control accuracy:1.000, Treat accuracy:1.000, overally accuray:1.000
Epoch: 0, Loss: 0.0000, Train error: 0.0000
At 1th fold testing: 


3 fold cross validation: 2it [00:35, 17.89s/it]

Control accuracy:1.000, Treat accuracy:1.000, overally accuray:1.000
Epoch: 0, Loss: 0.0000, Train error: 0.0000
At 2th fold testing: 


3 fold cross validation: 3it [00:53, 17.99s/it]
training at different percent:   0%|          | 0/1 [00:54<?, ?it/s]

Control accuracy:1.000, Treat accuracy:1.000, overally accuray:1.000


In [ ]:
sf_results = mini_noise_signal_cv(start = 90,
                                  end = 91,
                                  data = sf_drop_NA_data, 
                                  num_bag = 50,
                                  bag_size_mean = 200, 
                                  bag_size_std = 10, 
                                  treatment = "taxol", 
                                  control = "DMSO", 
                                  model = sf_deepset, 
                                  optimizer = optimizer,
                                  splits = 5,
                                  epochs = 2)

In [ ]:
sf_results

In [246]:
acc_control, acc_treat, pred_score_control, pred_score_treat = test(sf_deepset, dataloader)

In [29]:
(mean_control_accuracy, std_control_accuracy, 
            mean_treat_accuracy, std_treat_accuracy, 
            mean_pred_score_control, std_pred_score_control,
            mean_pred_score_treatment, std_pred_score_treatment) = sf_results

In [ ]:
dict = {'control_accuracy': nme, 'std_control_accuracy': deg, 'score': scr}  

In [24]:
sf_results = pd.DataFrame(list(sf_results)).transpose()

In [25]:
sf_results.columns = ["mean_control_accuracy", "std_control_accuracy", 
            "mean_treat_accuracy", "std_treat_accuracy", 
            "mean_pred_score_control", "std_pred_score_control",
            "mean_pred_score_treatment", "std_pred_score_treatment"]

In [26]:
sf_results

,mean_control_accuracy,std_control_accuracy,mean_treat_accuracy,std_treat_accuracy,mean_pred_score_control,std_pred_score_control,mean_pred_score_treatment,std_pred_score_treatment
0,0.6,0.489898,0.4,0.489898,0.392564,0.385677,0.393943,0.422387
1,0.1,0.300000,1.0,0.000000,0.722624,0.191698,0.814537,0.104709
2,0.4,0.489898,0.7,0.458258,0.543045,0.198318,0.673363,0.171835
3,0.9,0.300000,0.3,0.458258,0.213512,0.131968,0.393299,0.203889
4,0.9,0.300000,0.3,0.458258,0.187630,0.155302,0.295522,0.244676
5,0.7,0.458258,0.6,0.489898,0.324578,0.361401,0.679215,0.287370
6,0.8,0.400000,0.9,0.300000,0.343598,0.340132,0.728700,0.192789
7,1.0,0.000000,0.8,0.400000,0.134028,0.126734,0.733124,0.210020
8,0.6,0.489898,0.7,0.458258,0.386999,0.428574,0.691723,0.349524
9,0.7,0.458258,1.0,0.000000,0.397993,0.306416,0.989802,0.014643


In [24]:
class dmso_taxol_ProfileBag(D.Dataset):
    def __init__(self, df:pandas.DataFrame, size:int, bag_mean_size, bag_std_size,
                 perc:float, treatment:str, control:str, batch_size, merged_perc:float=0.5, transform:bool = False):
        batch_size = int(batch_size/2)
        self.df = df
        self.size = size
        self.batch_size = batch_size
        self.merged_pos_size = int(size) * batch_size
#         self.unmerged_neg_size = int(size * (1 - merged_perc)) * batch_size
        
        self.bag_mean_size = bag_mean_size
        self.bag_std_size = bag_std_size
        self.perc = perc
        self.treatment = treatment
        self.control = control
        self.bag_size = np.random.normal(bag_mean_size, bag_std_size, size).astype(int).repeat(batch_size)


        self.treatment_data = self.df[self.df["compound"] == self.treatment]
        self.control_data = self.df[self.df["compound"] == self.control]
        
        self.treatment_size = (self.bag_size * self.perc).astype(int)
        self.control_size = self.bag_size - self.treatment_size
        
        self.treatment_replace = len(self.treatment_data) < np.sum(self.treatment_size)
        self.control_replace = len(self.control_data) < np.sum(self.control_size)
        self.treat_index_list, self.control_index_list = self._get_sampling_index()
        
        self.transform = transform
        
    def _get_sampling_index(self):
        treat_index_list = []
        control_index_list = []

        treat_length = len(self.treatment_data)
        control_length = len(self.control_data)
        
        index = np.arange(treat_length)
        for i in range(self.merged_pos_size):
            if len(index) < self.treatment_size[i]:
                index = np.arange(treat_length)
            z = np.random.choice(index, self.treatment_size[i], replace=self.treatment_replace)
            index = index[~np.isin(index, z)]
            treat_index_list.append(z)

        index = np.arange(control_length)
        for i in range(self.merged_pos_size):
            if len(index) < self.control_size[i]:
                index = np.arange(control_length)
            z = np.random.choice(index, self.control_size[i], replace=self.control_replace )
            index = index[~np.isin(index, z)]
            control_index_list.append(z)
            
        return treat_index_list, control_index_list
        
    def _shuffle_tensor(self, mat):
        idxs = list(range(mat.shape[0]))
        random.shuffle(idxs)
        return mat[torch.tensor(idxs), :]    
    
    def __getitem__(self, index):
        index = index * self.batch_size
        X = []
        y = []
        for i in range(self.batch_size):
            merged_data = self.treatment_data.iloc[self.treat_index_list[index+i]].append(self.control_data.iloc[self.control_index_list[index+i]]) 
            sub_X, sub_y = data_label_split(merged_data.sample(frac=1))
            sub_X, sub_y = torch.from_numpy(sub_X.values), [torch.tensor([1.0]),sub_y]
            if self.transform == True:
                sub_X = self._shuffle_tensor(sub_X)
            X.append(sub_X) 
            y.append(sub_y)
            
        for i in range(self.batch_size):
            unmerged_bag_size = self.bag_size[index]
            sub_X, sub_y = data_label_split(self.control_data.sample(unmerged_bag_size))
            sub_X, sub_y = torch.from_numpy(sub_X.values), [torch.tensor([0.0]),sub_y]
            if self.transform ==  True:
                sub_X = self._shuffle_tensor(sub_X)
            X.append(sub_X)
            y.append(sub_y)
            
        X_shuf = []
        y_shuf = []
        index_shuf = list(range(len(X)))
        shuffle(index_shuf)
        for i in index_shuf:
            X_shuf.append(X[i])
            y_shuf.append(y[i])
            
        return torch.stack(X_shuf, dim = 0), y_shuf#torch.stack(y_shuf, dim = 0)
    

    def __len__(self):
        return self.size



In [16]:
from random import shuffle
import random

In [ ]:
start = 90
end = 91
data = mini_data
num_bag = 60
bag_size_mean = 200
bag_size_std = 10
treatment = "taxol"
control = "DMSO"
batch_size = 8
model = profile_AttSet(481).cuda()#FullDeepSet("mean").cuda()
splits = 3
epochs = 10

In [25]:
dataset = dmso_taxol_ProfileBag(mini_data, 10, 5, 1,
                 0.5, "taxol", "DMSO", 2, 0.5, False)

In [29]:
dataset[2][1]

[[tensor([1.]),
       compound
  1222     DMSO
  1300     DMSO
  2271    taxol
  2601     DMSO
  1541    taxol],
 [tensor([0.]),
       compound
  2038     DMSO
  1323     DMSO
  1891     DMSO
  1294     DMSO
  212      DMSO]]